<a href="https://colab.research.google.com/github/Emmanuel-K07/chatterbox/blob/master/stable/526_mix_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ComfyUI Installation + Keep-Alive pour Google Colab Pro
# Maintient la session active même quand vous fermez l'ordinateur

import subprocess
import sys
import os
from pathlib import Path
import requests
import time
import threading
from IPython.display import HTML, display, Javascript
import json

def run_cmd(cmd, check=True, capture=False):
    """Execute command with proper error handling"""
    print(f"🔧 Exécution: {cmd[:80]}...")
    result = subprocess.run(cmd, shell=True, capture_output=capture, text=True, executable='/bin/bash')
    if check and result.returncode != 0:
        print(f"❌ Erreur lors de: {cmd}")
        if capture and result.stderr:
            print(f"Erreur: {result.stderr}")
        return False
    return result.stdout if capture else True

# Keep-Alive system pour maintenir la session
def setup_keepalive():
    """Configure le système keep-alive pour éviter les déconnexions"""
    display(HTML('''
    <script>
    // Keep-alive system pour Colab
    let keepAliveInterval = null;
    let lastActivity = Date.now();

    function keepSessionAlive() {
        // Simule une activité utilisateur toutes les 5 minutes
        if (Date.now() - lastActivity > 300000) { // 5 minutes
            console.log('🔄 Keep-alive: maintien de la session...');

            // Envoie un ping discret au kernel
            if (window.google && window.google.colab && window.google.colab.kernel) {
                window.google.colab.kernel.proxyPort(8188, {'cache': false});
            }

            // Simule un mouvement de souris
            document.dispatchEvent(new MouseEvent('mousemove', {
                clientX: Math.random() * window.innerWidth,
                clientY: Math.random() * window.innerHeight
            }));

            lastActivity = Date.now();
        }
    }

    // Active le keep-alive toutes les 60 secondes
    if (keepAliveInterval) clearInterval(keepAliveInterval);
    keepAliveInterval = setInterval(keepSessionAlive, 60000);

    // Monitor l'activité réelle de l'utilisateur
    ['mousedown', 'mousemove', 'keypress', 'scroll', 'touchstart'].forEach(event => {
        document.addEventListener(event, () => lastActivity = Date.now(), true);
    });

    // Affichage du statut
    function updateStatus() {
        const statusElement = document.getElementById('keepalive-status');
        if (statusElement) {
            const minutesIdle = Math.floor((Date.now() - lastActivity) / 60000);
            statusElement.innerHTML = `🟢 Keep-alive actif | Inactif depuis: ${minutesIdle}min`;
        }
    }

    setInterval(updateStatus, 10000);
    console.log('✅ Keep-alive system activated');
    </script>

    <div id="keepalive-status" style="background: #e8f5e8; padding: 10px; border-radius: 5px; margin: 10px 0;">
        🟢 Keep-alive system: Activé
    </div>
    '''))

def create_connection_monitor():
    """Crée un moniteur de connexion pour suivre l'état du tunnel"""
    monitor_script = '''
    <div id="connection-monitor" style="background: #f0f8ff; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #4CAF50;">
        <h3>📡 Moniteur de Connexion ComfyUI</h3>
        <div id="tunnel-status">🔄 Vérification du tunnel...</div>
        <div id="comfyui-status">🔄 Vérification ComfyUI...</div>
        <div id="last-check">Dernière vérification: <span id="check-time">-</span></div>
        <button onclick="forceReconnect()" style="margin-top: 10px; padding: 8px 16px; background: #4CAF50; color: white; border: none; border-radius: 4px; cursor: pointer;">
            🔄 Force Reconnect
        </button>
    </div>

    <script>
    let monitorInterval = null;
    let tunnelUrl = null;

    async function checkServices() {
        const now = new Date().toLocaleTimeString();
        document.getElementById('check-time').textContent = now;

        try {
            // Vérification du tunnel Cloudflare
            const response = await fetch('/tmp/cloudflared.log');
            if (response.ok) {
                const logs = await response.text();
                const urlMatch = logs.match(/https:\\/\\/[^\\s]*trycloudflare\\.com/);
                if (urlMatch) {
                    tunnelUrl = urlMatch[0];
                    document.getElementById('tunnel-status').innerHTML = `✅ Tunnel actif: <a href="${tunnelUrl}" target="_blank">${tunnelUrl}</a>`;
                } else {
                    document.getElementById('tunnel-status').innerHTML = '⚠️ Tunnel: URL non trouvée';
                }
            } else {
                document.getElementById('tunnel-status').innerHTML = '❌ Tunnel: Logs inaccessibles';
            }

            // Test de connectivité ComfyUI
            if (tunnelUrl) {
                try {
                    const testResponse = await fetch(tunnelUrl + '/system_stats', {mode: 'no-cors'});
                    document.getElementById('comfyui-status').innerHTML = '✅ ComfyUI: Accessible';
                } catch (e) {
                    document.getElementById('comfyui-status').innerHTML = '⚠️ ComfyUI: Test de connectivité échoué';
                }
            } else {
                document.getElementById('comfyui-status').innerHTML = '⏳ ComfyUI: En attente du tunnel';
            }

        } catch (error) {
            console.error('Erreur lors de la vérification:', error);
        }
    }

    function forceReconnect() {
        console.log('🔄 Force reconnect...');
        // Réinitialise le tunnel
        fetch('/content/restart_tunnel.py', {method: 'POST'}).catch(e => console.log('Restart script not found'));
        setTimeout(checkServices, 5000);
    }

    // Vérification toutes les 30 secondes
    if (monitorInterval) clearInterval(monitorInterval);
    monitorInterval = setInterval(checkServices, 30000);
    checkServices(); // Première vérification immédiate
    </script>
    '''
    display(HTML(monitor_script))

print("🚀 Installation ComfyUI avec Keep-Alive pour Google Colab Pro")
print("=" * 60)

# Configuration du keep-alive AVANT l'installation
print("🔄 Configuration du système Keep-Alive...")
setup_keepalive()

# Votre script d'installation existant (identique)
print("📦 Préparation de l'environnement...")
os.chdir('/content')

# Installation PyTorch compatible
print("🔥 Installation PyTorch + CUDA 11.8...")
run_cmd('pip install --upgrade pip wheel setuptools')
run_cmd('pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118')

# Installation xformers compatible
print("⚡ Installation xformers optimisé...")
run_cmd('pip install xformers==0.0.22.post4 --index-url https://download.pytorch.org/whl/cu118')

# Résolution des conflits de versions
print("🔧 Résolution des conflits de dépendances...")
run_cmd('pip install numpy==1.24.3 --force-reinstall')
run_cmd('pip install pillow==10.0.1 --force-reinstall')
run_cmd('pip install opencv-python-headless==4.8.1.78')

# Clone ComfyUI
print("📥 Téléchargement ComfyUI...")
if os.path.exists('/content/ComfyUI'):
    run_cmd('rm -rf /content/ComfyUI')
run_cmd('git clone https://github.com/comfyanonymous/ComfyUI.git')
os.chdir('/content/ComfyUI')

# Installation dépendances ComfyUI
print("📚 Installation dépendances ComfyUI...")
run_cmd('pip install -r requirements.txt')
run_cmd('pip install scipy scikit-image transformers accelerate safetensors')

# Installation ComfyUI Manager
print("🔌 Installation ComfyUI Manager...")
manager_path = '/content/ComfyUI/custom_nodes/ComfyUI-Manager'
if os.path.exists(manager_path):
    run_cmd(f'rm -rf {manager_path}')
run_cmd(f'git clone https://github.com/ltdrdata/ComfyUI-Manager.git {manager_path}')

# Installation nodes essentiels (sans was-node-suite-comfyui qui cause des problèmes)
print("🔌 Installation des custom nodes essentiels...")
custom_nodes_dir = Path('/content/ComfyUI/custom_nodes')
custom_nodes_dir.mkdir(exist_ok=True)

essential_nodes = {
    'comfyui_controlnet_aux': 'https://github.com/Fannovel16/comfyui_controlnet_aux.git',
    'ComfyUI_IPAdapter_plus': 'https://github.com/cubiq/ComfyUI_IPAdapter_plus.git',
    'ComfyUI-AnimateDiff-Evolved': 'https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved.git',
    'ComfyUI-VideoHelperSuite': 'https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git',
    'ComfyUI-Impact-Pack': 'https://github.com/ltdrdata/ComfyUI-Impact-Pack.git'
}

for node_name, node_url in essential_nodes.items():
    node_path = custom_nodes_dir / node_name
    if node_path.exists():
        run_cmd(f'rm -rf {node_path}')
    print(f"📥 Clonage {node_name}...")
    run_cmd(f'git clone {node_url} {node_path}', check=False)

# Installation des dépendances spécifiques (évite les conflits)
print("🔧 Installation dépendances spécifiques...")
run_cmd('pip install insightface==0.7.3', check=False)
run_cmd('pip install onnxruntime-gpu', check=False)
run_cmd('pip install segment-anything', check=False)

# Création des dossiers de modèles
print("📁 Création des dossiers de modèles...")
models_base = Path('/content/ComfyUI/models')
model_dirs = [
    'checkpoints', 'controlnet', 'ipadapter', 'vae',
    'clip_vision', 'animatediff_models', 'upscale_models'
]

for model_dir in model_dirs:
    (models_base / model_dir).mkdir(parents=True, exist_ok=True)

# Configuration Cloudflare tunnel
print("🌐 Configuration du tunnel Cloudflare...")
if not os.path.exists('/content/cloudflared'):
    run_cmd('wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared')
    run_cmd('chmod +x /content/cloudflared')

# Script de lancement avec auto-restart
launch_script = '''#!/bin/bash
cd /content/ComfyUI

echo "🔄 Nettoyage des processus existants..."
pkill -f cloudflared 2>/dev/null || true
pkill -f "python main.py" 2>/dev/null || true
sleep 3

# Fonction de redémarrage automatique
restart_services() {
    echo "🔄 Redémarrage des services..."
    pkill -f cloudflared 2>/dev/null || true
    sleep 2

    echo "🌐 Redémarrage du tunnel Cloudflare..."
    nohup /content/cloudflared tunnel --url http://127.0.0.1:8188 > /tmp/cloudflared.log 2>&1 &
    sleep 8

    # Vérification et affichage de l'URL
    TUNNEL_URL=$(grep -o 'https://[^[:space:]]*trycloudflare.com' /tmp/cloudflared.log 2>/dev/null | head -1)
    if [ ! -z "$TUNNEL_URL" ]; then
        echo "✅ Tunnel restauré: $TUNNEL_URL"
        echo "$TUNNEL_URL" > /tmp/tunnel_url.txt
    fi
}

# Démarrage initial
echo "🌐 Démarrage du tunnel Cloudflare..."
nohup /content/cloudflared tunnel --url http://127.0.0.1:8188 > /tmp/cloudflared.log 2>&1 &
sleep 8

# Récupération de l'URL
TUNNEL_URL=""
for i in {1..15}; do
    TUNNEL_URL=$(grep -o 'https://[^[:space:]]*trycloudflare.com' /tmp/cloudflared.log 2>/dev/null | head -1)
    if [ ! -z "$TUNNEL_URL" ]; then
        echo "✅ Tunnel actif: $TUNNEL_URL"
        echo "$TUNNEL_URL" > /tmp/tunnel_url.txt
        break
    fi
    echo "⏳ Recherche URL tunnel... ($i/15)"
    sleep 2
done

if [ -z "$TUNNEL_URL" ]; then
    echo "❌ Impossible de récupérer l'URL du tunnel initialement"
else
    echo "🎉 ComfyUI sera accessible sur: $TUNNEL_URL"
fi

# Démarrage de ComfyUI avec auto-restart
echo "🚀 Démarrage de ComfyUI avec auto-restart..."
while true; do
    python main.py --listen 0.0.0.0 --port 8188 --dont-print-server 2>&1 | tee -a /tmp/comfyui.log

    echo "⚠️ ComfyUI s'est arrêté. Redémarrage dans 10 secondes..."
    sleep 10

    # Vérification et redémarrage du tunnel si nécessaire
    if ! pgrep -f cloudflared > /dev/null; then
        restart_services
    fi
done
'''

with open('/content/launch_comfyui.sh', 'w') as f:
    f.write(launch_script)
run_cmd('chmod +x /content/launch_comfyui.sh')

# Script de restart tunnel
restart_script = '''#!/usr/bin/env python3
import subprocess
import time
import os

def restart_tunnel():
    print("🔄 Redémarrage du tunnel...")
    subprocess.run(['pkill', '-f', 'cloudflared'], capture_output=True)
    time.sleep(3)

    subprocess.Popen(['/content/cloudflared', 'tunnel', '--url', 'http://127.0.0.1:8188'],
                    stdout=open('/tmp/cloudflared.log', 'w'),
                    stderr=subprocess.STDOUT)
    print("✅ Tunnel redémarré")

if __name__ == "__main__":
    restart_tunnel()
'''

with open('/content/restart_tunnel.py', 'w') as f:
    f.write(restart_script)
run_cmd('chmod +x /content/restart_tunnel.py')

print("✅ Installation terminée avec Keep-Alive!")
print("=" * 60)

# Affichage du moniteur de connexion
create_connection_monitor()

print()
print("🚀 LANCEMENT PERSISTANT:")
print("   Dans une nouvelle cellule:")
print("   !bash /content/launch_comfyui.sh")
print()
print("💡 AVANTAGES KEEP-ALIVE:")
print("   ✅ Session maintenue quand vous fermez l'ordinateur")
print("   ✅ Auto-restart si ComfyUI plante")
print("   ✅ Moniteur de connexion en temps réel")
print("   ✅ Redémarrage automatique du tunnel")
print()
print("⚠️  IMPORTANT:")
print("   • Gardez cet onglet ouvert en arrière-plan")
print("   • Le keep-alive fonctionne même si l'écran se met en veille")
print("   • Durée max: ~12h sur Colab Pro (24h sur Pro+)")

In [ ]:
!bash /content/launch_comfyui.sh


In [ ]:
!bash /content/check_status.sh